In [213]:
import cadquery as cq
from jupyter_cadquery import (
    PartGroup, Part, Edges, Faces, Vertices, show,
    close_viewer, close_viewers, get_viewer, open_viewer, set_defaults, get_defaults
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay, reset_replay

set_defaults(axes=True, timeit=False)

cv = open_viewer("Examples", cad_width=640, height=480, glass=True)

enable_replay(False, False)
show_object = replay


Enabling jupyter_cadquery replay


In [216]:
def mounting_box(l, w, h, wall_thickness=1, frac=2):
    """
    Creates a mounting "box"    
    """
    wall_thickness *= 2
    oshell = (
        cq.Workplane("XY")
        .workplane(offset=h/2)
        .box(l+wall_thickness, w+wall_thickness, h)
    )
    ishell = (
        cq.Workplane("XY")
        .workplane(offset=h/2)
        .box(l, w, h)
    )
    box = oshell.cut(ishell)

    if frac > 0:
        box = box.faces("<Y").workplane(centerOption="CenterOfMass").rect(l/frac, h).cutThruAll()
        box = box.faces("<X").workplane(centerOption="CenterOfMass").rect(w/frac, h).cutThruAll()
    
    return box

def pin_stands(l, w, outer, inner, height=2):
    thick_pins = (
        cq.Workplane("XY")
        .rect(l, w, forConstruction=True)
        .vertices()
        .circle(outer)
        .extrude(height)
        .faces(">Z")
        .workplane()
        .rect(l, w, forConstruction=True)
        .vertices()
        .circle(inner)
        .extrude(height)      
    )
    
    return thick_pins
    
    
def transistor_holder(diameter, height=2, padding=1):
    radius = diameter / 2
    holder = (
        cq.Workplane("XY")
        .circle(radius + padding)
        .extrude(height)
        .faces("<Z")
        .circle(radius)
        .cutThruAll()
        
    )
    return holder


def snap_join(l, w, radius=0.5):
    snap_join = (
        cq.Workplane("top")
        .circle(radius)
        .extrude(w)
        .copyWorkplane(
            cq.Workplane("left", origin=(0, 0, 0))
        )
        .circle(radius)
        .extrude(l)
        .copyWorkplane(
            cq.Workplane("left", origin=(0, w, 0))
        )
        .circle(radius)
        .extrude(l)
        .copyWorkplane(
            cq.Workplane("top", origin=(-l, 0, 0))
        )
        .circle(radius)
        .extrude(w)
    )
    return snap_join
        

def container(base_length, base_width, height, wall_thickness=1, with_snap=True):
    height = height + wall_thickness
    sides = wall_thickness * 2
    oshell = (
        cq.Workplane("XY")
        .workplane(height/2 - wall_thickness)
        .box(base_length+sides, base_width+sides, height)
        .edges("|Z").fillet(2)
    )

    ishell = (
        oshell.faces("<Z")
        .workplane(invert=True, offset=wall_thickness)
        .rect(base_length, base_width)
        .extrude(height, False)
    )
    
    container = oshell.cut(ishell)
        
    if with_snap:
        snap_pos = height - wall_thickness -2
        grove = (
            snap_join(base_length, base_width)
            .translate((base_length/2, -base_width/2, snap_pos))
        )
        container = container.cut(grove)
    
    

    return container

def lid(base_length, base_width, height, wall_thickness=2, with_snap=True):
    ct = (
        cq.Workplane("XY")
        .workplane(height/2 - wall_thickness)
        .box(base_length+wall_thickness, base_width+wall_thickness, height)
        .edges("|Z").fillet(2)
    )

    fuzz = 0.75
    
    ct = (
        ct.faces(">Z")
        .rect(base_length-fuzz, base_width-fuzz)
        .extrude(3)
    )
    
    ct = (
        ct.faces(">Z")
        .rect(base_length-fuzz-wall_thickness, base_width-fuzz-wall_thickness)
        .cutBlind(-height-2)
    )
    
    
    if with_snap:
        snap_pos = height
        grove = (
            snap_join(base_length-fuzz, base_width-fuzz)
            .translate(((base_length-fuzz)/2, -(base_width-fuzz)/2, snap_pos))
        )
        
        cut_out = (
            cq.Workplane("XY")
            .rect(base_length, base_width)
            .vertices()
            .box(base_length/2, base_width/2, 2)
            .translate((0, 0, snap_pos))
        )
        grove = grove.cut(cut_out)
        
        ct = ct.union(grove)

    
    return ct


In [5]:
pico_holder = mounting_box(53, 23, 5)
relay_holder = mounting_box(44, 18, 5)
pins = pin_stands(47, 11.4, 2, .8)


boiler_base = (
    cq.Assembly(name="case")
    .add(pico_holder, name="pico")
    .add(pins, name="pico-pins", color=cq.Color("orange"))
    .add(relay_holder, name="relay", color=cq.Color("red"), loc=cq.Location(cq.Vector((53-44)/2,30,0)))
    .add(transistor_holder(5), name="transistor", color=cq.Color("blue"), loc=cq.Location(cq.Vector(-25, 30, 0)))
    .add(container(70, 70, 4), loc=cq.Location(cq.Vector(0, 15, 0)))
)   


boiler_base

100% ⋮————————————————————————————————————————————————————————————⋮ (5/5)  0.13s


In [555]:
cq.exporters.export(boiler_base.toCompound(), 'boiler-base.stl')


# Thermostats

In [556]:
pico_holder = mounting_box(53, 23, 5)
sensor_holder = mounting_box(20, 16, 3)


thermo_base = (
    cq.Assembly(name="case")
    .add(pico_holder, name="pico")
    .add(pins, name="pico-pins", color=cq.Color("orange"))
    .add(sensor_holder, name="relay", color=cq.Color("red"), loc=cq.Location(cq.Vector(50, 0, 0)))
#     .add(transistor_holder(5), name="transistor", color=cq.Color("blue"), loc=cq.Location(cq.Vector(-25, 30, 0)))
    .add(container(100, 35, 4), loc=cq.Location(cq.Vector(20, 0, 0)))
)   


thermo_base

100% ⋮————————————————————————————————————————————————————————————⋮ (4/4)  0.08s


In [557]:


snap_join(20, 50)

In [558]:
cq.exporters.export(container(10, 15, 5), 'container.stl')


In [ ]:
relay_lid = (
    lid(70, 70, 40, with_snap=False)
    .faces("<Z")
    .workplane()
    .text("Pico\nTherm", 15, -1)
)
cq.exporters.export(relay_lid, 'relay_lid.stl')
relay_lid

# Box with buttons

In [ ]:
def add_button_holder(box, x, y):
    # Fixed size 7x7mm with 3.8mm hole
    dim = 7
    dia = 3.8
    height = 5
    
    holder = mounting_box(dim, dim, height, frac=10)
    
    box = box.faces(">Z").center(x, y).hole(dia).center(-x, -y)
    box = box.union(holder.translate((x, y, 0)))
    
    return box

def add_lrd_holder(box, x, y):
    # Fixed size 5.25mm holder with 4mm hole
    holder = 5.25
    dia = 4
    
    holder = transistor_holder(holder, height=4)
    
    box = box.faces(">Z").center(x, y).hole(dia).center(-x, -y)
    box = box.union(holder.translate((x, y, 0)))
    
    return box

    


# button_holder = mounting_box(7, 7, 5, frac=99)

pico_holder = mounting_box(53, 23, 5)
pins = pin_stands(47, 11.4, 2, .8)


box = container(60, 60, 15, wall_thickness=2)

but_x, but_y = -15, -15
box = add_button_holder(box, but_x+0, but_y+0)
box = add_button_holder(box, but_x+10, but_y+0)
box = add_button_holder(box, but_x+-10, but_y+0)
box = add_button_holder(box, but_x+0, but_y+-10)
box = add_button_holder(box, but_x+0, but_y+10)

box = add_lrd_holder(box, -but_x, but_y)

box = box.faces(">X").workplane().center(15, 0).rect(20, 10).cutBlind("next")
box = box.faces(">X").workplane(centerOption="CenterOfBoundBox").center(-15, 0).circle(5).cutBlind("next")

button_holder = (
    cq.Assembly(name="case")
    .add(box, name="box")
    .add(pico_holder, name="pico", loc=cq.Location(cq.Vector(-3, 15, 0)))
    .add(pins, name="pins", loc=cq.Location(cq.Vector(-3, 15, 0)))

)

button_holder = button_holder.toCompound()

cq.exporters.export(button_holder, 'julian_light_holder.stl')

button_holder

In [ ]:
box.faces(">X").translate((0,15, -5)).rect(20, 20, forConstruction=True)

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.00s


In [218]:
my_lid = (
    lid(60, 60, 10)
)
cq.exporters.export(my_lid, 'my_lid.stl')
my_lid